In [1]:
# scripts/detect_issues.py
import json
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()
INPUT_JSON = "processed/extracted_info.json"
OUTPUT_ISSUES = "processed/detected_issues.json"

# Cargar extracción
with open(INPUT_JSON, "r", encoding="utf-8") as f:
    data = json.load(f)

# Secciones obligatorias que queremos asegurar
REQUIRED_SECTIONS = [
    "clausulas_legales",
    "requisitos_tecnicos",
    "condiciones_economicas",
    # puedes añadir: "garantias", "plazos", "multas", etc.
]

# Detectar vacíos simples
vacios = {}
for sec in REQUIRED_SECTIONS:
    val = data.get(sec, "")
    if not val or (isinstance(val, str) and val.strip() == ""):
        vacios[sec] = "Falta o está vacío"

# Usa LLM para detectar ambigüedades o contradicciones (opcional)
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
prompt = f"""
Eres un auditor de contratos. Revisa el siguiente contenido (puede estar fragmentado) y
devuelve un JSON con:
- ambiguedades: lista de frases o cláusulas que son ambiguas o confusas.
- contradicciones: lista de campos que se contradicen entre sí (por ejemplo, 'no hay multa' y 'multas: sí').
Texto: {json.dumps(data, ensure_ascii=False)}
"""

resp = llm.invoke(prompt).content  # depende de tu versión; ajusta si usas LLMChain
# Limpieza simple: intentar parsear JSON si devuelve; si no, guardar como texto.
issues = {"vacios": vacios, "llm_output": resp}

os.makedirs(os.path.dirname(OUTPUT_ISSUES), exist_ok=True)
with open(OUTPUT_ISSUES, "w", encoding="utf-8") as f:
    json.dump(issues, f, ensure_ascii=False, indent=2)

print("Issues detectados guardados en", OUTPUT_ISSUES)

Issues detectados guardados en processed/detected_issues.json
